##  GEE STAC Catalog

Google Earth Engine produces a static catalogUse `pystac_client` package to query a static STAC catalog as a JSON file on Google Cloud Storage.

We use PySTAC Client to read and extract information from this catalog.
 
PySTAC Client (`pystac_client`) builds upon PySTAC (`pystac`) through higher-level functionality. Some of the functions are inherited from the `pystac` library

In [ ]:
try:
    import pystac_client
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !apt install pystac_client -qq
    else:
        print('pystac_client not found, please install via conda in your environment')

In [ ]:
from pystac_client import Client
import pandas as pd

## Catalog

A STAC Catalog is a top-level object. Create a STAC Catalog object from the `catalog.json`.

In [ ]:
catalog = Client.open('https://earthengine-stac.storage.googleapis.com/catalog/catalog.json')

Check the catalog description. The description is in Markdown format, so we format it to HTML.

In [ ]:
from IPython.display import Markdown
Markdown(catalog.description)

There are [many types](https://pystac.readthedocs.io/en/stable/concepts.html#catalog-types) of STAC Catalogs. We check the type of this catalog.

In [ ]:
catalog.catalog_type

## Collection

A STAC Collection is used to group related Items and provide aggregate or summary metadata for those Items.

STAC Catalogs may have many nested layers of Catalogs or Collections within the top-level collection.

The GEE Catalog contains nested collections, grouped by providers. Here we query for all collections by provider 'USDA'

In [ ]:
cols = catalog.get_child('USDA')
cols

We can get a child collection by specifying the id. Since this is a nested collection, we specify `recursive=True`

In [ ]:
col = catalog.get_child('USDA/NAIP/DOQQ', recursive=True)
col

The result is a `CollectionClient` object. Let's extract the bsaic information about this collection.

In [ ]:
col.title

In [ ]:
Markdown(col.description)

A catalog consists of one or more collections. Let's get all the collections from the catalog and iterate through them.

The collection also has some GEE specific metadata

In [ ]:
Markdown(col.extra_fields['gee:terms_of_use'])

STAC collections have extensions that allows one to specify additional domain specific information. Most remote sensing datasets have fields from the [Electro-Optical (EO)](https://github.com/stac-extensions/eo) extension that can describe the images, bands and their spatial resolutions.

In [ ]:
summaries = col.summaries.to_dict()

summaries['eo:bands']

## Collections

We can query the catalog for all collections instead of specifying a single colleciton by id.

In [ ]:
collections = catalog.get_all_collections()

The `collections` is a generator object. You can iterate over it and extract information fom each [STAC Collection](https://pystac.readthedocs.io/en/stable/api/collection.html) object.

In [ ]:
datasets = []

for col in catalog.get_all_collections():
    datasets.append({
        'id': col.id,
        'title': col.title,
        'description':col.description
    })

Remove all deprecated collections.

In [ ]:
datasets = [d for d in datasets if 'deprecated' not in d['title']]

Create a Pandas DataFrame.

In [56]:
df = pd.DataFrame(datasets)
df

,id,title,description
0,AAFC/ACI,Canada AAFC Annual Crop Inventory,"Starting in 2009, the Earth Observation Team o..."
1,ACA/reef_habitat/v1_0,Allen Coral Atlas (ACA) - Geomorphic Zonation ...,The [Allen Coral Atlas](https://allencoralatla...
2,AHN/AHN2_05M_INT,"AHN Netherlands 0.5m DEM, Interpolated",The AHN DEM is a 0.5m DEM covering the Netherl...
3,AHN/AHN2_05M_NON,"AHN Netherlands 0.5m DEM, Non-Interpolated",The AHN DEM is a 0.5m DEM covering the Netherl...
4,AHN/AHN2_05M_RUW,"AHN Netherlands 0.5m DEM, Raw Samples",The AHN DEM is a 0.5m DEM covering the Netherl...
...,...,...,...
580,YALE/YCEO/UHI/UHI_yearly_pixel/v4,YCEO Surface Urban Heat Islands: Pixel-Level A...,"This dataset contains annual, summertime, and ..."
581,YALE/YCEO/UHI/Winter_UHI_yearly_pixel/v4,YCEO Surface Urban Heat Islands: Pixel-Level Y...,"This dataset contains annual, summertime, and ..."
582,projects/planet-nicfi/assets/basemaps/africa,Planet & NICFI Basemaps for Tropical Forest Mo...,This image collection provides access to high-...
583,projects/planet-nicfi/assets/basemaps/americas,Planet & NICFI Basemaps for Tropical Forest Mo...,This image collection provides access to high-...


Save to Excel.

In [ ]:
df.to_excel('ee_datasets.xlsx', index=False)